<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Worrying-About-Overfitting" data-toc-modified-id="Worrying-About-Overfitting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Worrying About Overfitting</a></span><ul class="toc-item"><li><span><a href="#Use-Train-Validation-Test" data-toc-modified-id="Use-Train-Validation-Test-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Use Train-Validation-Test</a></span></li><li><span><a href="#Model-Complexity-Graph" data-toc-modified-id="Model-Complexity-Graph-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Model Complexity Graph</a></span><ul class="toc-item"><li><span><a href="#Early-Stopping" data-toc-modified-id="Early-Stopping-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Early Stopping</a></span></li></ul></li></ul></li><li><span><a href="#When-a-Good-Model-Goes-Bad" data-toc-modified-id="When-a-Good-Model-Goes-Bad-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>When a Good Model Goes Bad</a></span><ul class="toc-item"><li><span><a href="#L1-Regularization---Absolute-Value" data-toc-modified-id="L1-Regularization---Absolute-Value-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>L1 Regularization - Absolute Value</a></span></li><li><span><a href="#L2-Regularization---Squared-Value" data-toc-modified-id="L2-Regularization---Squared-Value-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>L2 Regularization - Squared Value</a></span></li></ul></li><li><span><a href="#Dropout" data-toc-modified-id="Dropout-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Dropout</a></span><ul class="toc-item"><li><span><a href="#Avoiding-the-Self-Perpetuating-Strength-Training" data-toc-modified-id="Avoiding-the-Self-Perpetuating-Strength-Training-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Avoiding the Self-Perpetuating Strength Training</a></span></li><li><span><a href="#Example-Code" data-toc-modified-id="Example-Code-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Example Code</a></span></li></ul></li></ul></div>

# Worrying About Overfitting

A big issue is making sure we don't overfit our model

## Use Train-Validation-Test

- Think of **training** as what you study for a test
- Think of **validation** is using a practice test
- Think of **testing** as what you use to judge the model 

> ***holdout*** is when your test dataset is never used for training (unlike in cross-validation)

## Model Complexity Graph

- Underfitting
    + low complexity --> high bias, low variance
    + training error: large
    + testing error: large
- Overfitting
    + high complexity --> low bias, high variance
    + training error: low
    + testing error: large

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

test_error = np.array([5,3.5,2,3,4])
train_error = np.array([4.5,3,1.5,1,0.5])
n_epochs = np.array([5,50,100,200,300])

plt.scatter(n_epochs, train_error,)
plt.scatter(n_epochs, test_error)
plt.legend(['train error','test error'])
plt.xlabel('Number of Epochs')
plt.ylabel('Error')
plt.show()

### Early Stopping 

We can stop our training early when our test error stopped dropping

# When a Good Model Goes Bad

When a model has large weights, the model is "too confident"

We need to punish large (confident) weights by contributing them to the error function

## L1 Regularization - Absolute Value

- Tend to get sparse vectors (small weights go to 0)
- Reduce number of weights
- Good feature selection to pick out importance

$$ J(W,b) = -\dfrac{1}{m} \sum^m_{i=1}\big[\mathcal{L}(\hat y_i, y_i)+ \dfrac{\lambda}{m}|w_i| \big]$$


## L2 Regularization - Squared Value

- Not sparse vectors (weights homogeneous & small)
- Gives better results for training
    + subtle; consider vectors: [1,0] & [0.5, 0.5] 
    + recall we want smallest value for our value
    + L2 prefers [0.5,0.5] over [1,0] 
    
$$ J(W,b) = -\dfrac{1}{m} \sum^m_{i=1}\big[\mathcal{L}(\hat y_i, y_i)+ \dfrac{\lambda}{m}w_i^2 \big]$$

# Dropout

You want to even out your workouts, otherwise you may have some strange results...

<img src='images/homer-dropout-comparison.jpg'/>

Well, our neural network models are the same way. The model should get _evenly_ trained. We don't want to train the same node/pathway over and over again

## Avoiding the Self-Perpetuating Strength Training

When working out, we'd train our left and right arms evenly and switch our exercise routine throughout the week.

In neural networks, we switch around which nodes we use during our training.

Assign a probability of using a given node for that epoch (usually about 20% chance). When we have many epochs, we likely will even out the randomness

<img src='images/layered-neural-net.jpg'/>

## Example Code

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Dropout

n_classes = 10

model = Sequential()

# Input Layer
model.add(Dense(128, input_dim=100, activation='relu', name='input_layer'))
model.add(Dropout(0.2, name='input_dropout'))
# Hidden Layer
model.add(Dense(256, input_dim=100, activation='relu', name='hidden_layer1'))
model.add(Dropout(0.2, name='hidden_layer1_dropout'))
# Output Layer
model.add(Dense(n_classes, activation='softmax', name='output'))


model.summary()